# Load the csv file and the labels

In [4]:
#| code-fold: true
#read data file

import pandas as pd
import numpy as np
df = pd.read_csv("Oct22_FR_BP_reduced.csv",index_col=[0],float_precision=None, low_memory=False)
weighted = pd.read_csv("Oct22_FR_weighted.csv",index_col=[0],float_precision=None)
bp_labels = pd.read_csv("Oct22_FR_BP_reduced_labels.csv",index_col=[0])


# Merge BP data with weighted file

In [5]:
#| code-fold: true
wtd_cols = [col for col in weighted if col not in df.columns]
bp_wtd_gal19 = df.merge(weighted[wtd_cols], how="outer", on="Respondent_Serial")

# Helpers functions :  identify brands, identify metric and recode into T2B

In [6]:
#| code-fold: true
def identify_brand(column,category,market):

    if category == "corporate":
            if "_1_" in column:
                brand = "Google"
            elif "_2_" in column:
                brand = "Apple"
            elif "_3_" in column:
                brand = "Facebook"
            elif "_4_" in column:
                brand = "Microsoft"
            elif "_5_" in column:
                brand = "Amazon"
            elif "_6_" in column:
                brand = "YouTube"
    return brand

def identify_metric(column,category):

    if category == "corporate": 
        if "Q0301" in column:
            metric = "AWARENESS"
        elif "Q0302" in column:
            metric = "FAVORABILITY"
        elif "Q0307" in column:
            metric = "ADVOCACY"
        elif "Q0308" in column:
            metric = "TRUST"
        elif "funcEmotAttr_helpful_scale" in column:
            metric = "HELPFUL"
    return metric

def recode_6(col):
    if col in [1,2,3,4]:
        return 0
    elif col in [5,6]:
        return 1
    else:
        return col

def filtering(galbp, category):

    if category == "corporate":
        mask = (galbp["H_sample01"] == 1)
    galbp = galbp[mask].copy()

    return galbp


# init

In [7]:
#| code-fold: true
mycols=['AGEGEN_FOR_WGT','EDUX_FOR_WGT','REG_FOR_WGT','OS_FOR_WGT']
 
all_columns = {
    "corporate" : [
        "Q0301_awafam_brand_1_Q0301_awafam_brand_scale", 
        "Q0301_awafam_brand_2_Q0301_awafam_brand_scale"]}


# compute function

In [8]:
#| code-fold: true
def compute(column_bp, category, market):
    bp = filtering(bp_wtd_gal19, category)
    brand = identify_brand(column_bp, category, market)
    metric = identify_metric(column_bp, category)
    name = metric + "_" + brand 
    if metric == "AWARENESS":
        bp[column_bp]=bp[column_bp].apply(recode_6)
        n_levels = 2
        levels = ["0 - No", "1 - Top2Box"]
    
        ft = pd.DataFrame({"Metric": [name]*n_levels, "Levels": levels})
    ft["level"] = ft["Levels"].map(lambda x: int(x[0]))

    ft.set_index("Metric", inplace=True)
    ft["Frequency_GAL19"] = ft["level"].map(lambda x: bp[bp[column_bp] == x]["weight1"].sum() if pd.notnull(x) \
                                            else bp[bp[column_bp].isnull()]["weight1"].sum())
    ft["Wtd_Rel_Freq_GAL19"] = ft["Frequency_GAL19"].map(lambda x: x / ft["Frequency_GAL19"].sum())
  
    unwgt = bp.groupby(mycols)[column_bp].agg(['count','mean'])
    unwgt = unwgt.reset_index()
    unwgt['Metric'] = name
    unwgt.set_index('Metric', inplace=True)
    return ft, unwgt
    
    

# Main function

In [9]:
#| code-fold: true
def main(category, market):

    long_table = pd.DataFrame({"Metric": [], 
                            "Levels": [], 
                            "Frequency_GAL19": [], 
                            "Wtd_Rel_Freq_GAL19": []}).set_index("Metric")
    long_unwgt = pd.DataFrame({"AGEGEN_FOR_WGT": [], 
                            "EDUX_FOR_WGT": [], 
                            "REG_FOR_WGT": [], 
                            "OS_FOR_WGT": []})
    for column_bp in all_columns[category]:
        ft, unwgt = compute(column_bp, category, market)
        long_table = pd.concat([long_table, ft], axis=0)
        long_unwgt = pd.concat([long_unwgt, unwgt], axis=0)

    return long_table, long_unwgt

long_table, long_unwgt = main("corporate", "FR")


In [10]:
long_table


,Levels,Frequency_GAL19,Wtd_Rel_Freq_GAL19,level
Metric,,,,
AWARENESS_Google,0 - No,681.323764,0.242464,0.0
AWARENESS_Google,1 - Top2Box,2128.676263,0.757536,1.0
AWARENESS_Apple,0 - No,1623.378990,0.577715,0.0
AWARENESS_Apple,1 - Top2Box,1186.621037,0.422285,1.0


In [11]:

long_unwgt


,AGEGEN_FOR_WGT,EDUX_FOR_WGT,REG_FOR_WGT,OS_FOR_WGT,count,mean
AWARENESS_Google,"[16,25)F",High,ARA,Android,1.0,1.0
AWARENESS_Google,"[16,25)F",High,ARA,iOS,2.0,0.5
AWARENESS_Google,"[16,25)F",High,BFC,Android,1.0,1.0
AWARENESS_Google,"[16,25)F",High,BFC,iOS,1.0,0.0
AWARENESS_Google,"[16,25)F",High,BRT,Android,1.0,1.0
...,...,...,...,...,...,...
AWARENESS_Apple,"[55,Inf]M",Mid,PdL,Android,10.0,0.0
AWARENESS_Apple,"[55,Inf]M",Mid,PdL,iOS,3.0,1.0
AWARENESS_Apple,"[55,Inf]M",PNA,IdF,Android,1.0,0.0
AWARENESS_Apple,"[55,Inf]M",PNA,PAC,Other/DK,1.0,0.0
